In [108]:
# Run this cell to see if things work
import sys

import torch
from torch.nn import Parameter
import torch.nn as nn
import torch.nn.functional as F

import syft as sy
hook = sy.TorchHook(torch)

torch.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

In [109]:
x = torch.tensor([1,2,3,4,5])
y = x + x
print(y)

tensor([ 2,  4,  6,  8, 10])


We want to perform some computation on some other machine. We can no longer simply assume that the data is on our local machine. Thus, instead of using Torch tensors, we're now going to work with pointers to tensors. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [110]:
bob = sy.VirtualWorker(hook, id="bob")

In [111]:
x = torch.tensor([1,2,3,4,5])
y = torch.tensor([1,1,1,1,1])

Send these sensors to Bob

When we called x.send(bob) it returned a new object that we called x_ptr. This is our first pointer to a tensor. Pointers to tensors do NOT actually hold data themselves. Instead, they simply contain metadata about a tensor (with data) stored on another machine.

There are two main attributes specific to pointers:

x_ptr.location : bob, the location, a reference to the location that the pointer is pointing to

x_ptr.id_at_location : random integer, the id where the tensor is stored at location

In [112]:
x_ptr = x.send(bob)
y_ptr = y.send(bob)

x_ptr

(Wrapper)>[PointerTensor | me:13350006903 -> bob:91853869541]

The local worker which owns the pointer is also a VirtualWorker, although we didn't create it. Fun fact, just like we had a VirtualWorker object for Bob, we (by default) always have one for us as well. This worker is automatically created when we called hook = sy.TorchHook()

In [113]:
me = sy.local_worker
me == x_ptr.owner

True

Let's check Bob Sensors

In [114]:
#bob._objects

In [115]:
z = x_ptr + y_ptr
z.get()

tensor([2, 3, 4, 5, 6])

In [116]:
z= torch.add(x_ptr,y_ptr)
z.get()

tensor([2, 3, 4, 5, 6])

In [117]:
x = torch.tensor([1,2,3,4,5.], requires_grad=True).send(bob)
y = torch.tensor([1,1,1,1,1.], requires_grad=True).send(bob)
z = (x + y).sum()
z.backward()
x.get().grad

tensor([1., 1., 1., 1., 1.])